In [14]:
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import tensorflow as tf
from sklearn.metrics import accuracy_score

In [15]:
def cal_iou(y_true, y_pred):
    x1 = max(y_true[0], y_pred[0])
    y1 = max(y_true[1], y_pred[1])
    x2 = min(y_true[2], y_pred[2])
    y2 = min(y_true[3], y_pred[3])

    intersection_area = max(0, x2 - x1 + 1) * max(0, y2 - y1 + 1)

    true_area = (y_true[2] - y_true[0] + 1) * (y_true[3] - y_true[1] + 1)
    bbox_area = (y_pred[2] - y_pred[0] + 1) * (y_pred[3] - y_pred[1] + 1)

    iou = intersection_area / float(true_area + bbox_area - intersection_area)
    return iou

In [16]:
global height
global width
global num_classes
global label_names
np.random.seed(42)
tf.random.set_seed(42)
height = 256
width = 320
mean_iou = []
pred_labels = []

In [17]:
# Load Labels and Return Names of Classes
def load_labels(path):
    df = pd.read_csv(os.path.join(path,"names.csv"),header=None) # Reading the csv file
    print(df.head())
    names = df[0].tolist() #convert it to list
    return names

In [19]:
classes = load_labels("Stanford Car Dataset/")
num_classes = len(classes)

                            0
0  AM General Hummer SUV 2000
1         Acura RL Sedan 2012
2         Acura TL Sedan 2012
3        Acura TL Type-S 2008
4        Acura TSX Sedan 2012


In [20]:
model_path = os.path.join("files", "model.h5")

In [21]:
model = tf.keras.models.load_model(model_path)

In [24]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _, image = cap.read()
    x = cv2.resize(image, (width, height))
    x = (x - 127.5) / 127.5
    x = np.expand_dims(x, axis=0)



    """ Prediction """
    pred_bbox, label = model.predict(x, verbose=0)
    pred_bbox = pred_bbox[0]
    label_index = np.argmax(label[0])
    pred_labels.append(label_index+1)

    """ Rescale the bbox points. """
    pred_x1 = int(pred_bbox[0] * image.shape[1])
    pred_y1 = int(pred_bbox[1] * image.shape[0])
    pred_x2 = int(pred_bbox[2] * image.shape[1])
    pred_y2 = int(pred_bbox[3] * image.shape[0])


    """ Plot bbox on image """
    image = cv2.rectangle(image, (pred_x1, pred_y1), (pred_x2, pred_y2), (0, 0, 255), 2) ## RED

    """ Plot predicted class label and score """
    font_size = 1
    pred_class_name = classes[label_index]
    text = f"{pred_class_name}"
    cv2.putText(image, text, (pred_x1, pred_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 255), 2)

    """ Saving the image """
    cv2.imshow('EyeTrack', image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()